# BUMP data: long, multivariate time series with missing values

For each participant, we have a large number of variables recorded through the course of pregnancy. For instance, we may have one person's weight measured by their [BodyPort Smart Scale](https://www.bodyport.com/) every few days for a few months along with nighttime sleep data from their [Oura Smart Ring](https://ouraring.com/) recorded for most days. The participants also fill out different surveys at different times about their medical history, life events, ongoing depression/anxiety scores, and much more. Altogether, these data are challenging to learn from since most measures are not made at the same time, so heavy preprocessing is required if we want to use traditional, well-established techniques.

In this document, we include descriptions and examples for different types of preprocessing that are useful for the BUMP study.

## Missing Values

Missing values plague digital health, and there are many reasons a value may be "missing". For example:
* A survey score was not reported on one day because the participant **wasn't asked** to fill out the survey (predictable missingness)
* A survey score was not reported on one day because the participant **forgot** to fill out the survey (unpredictable missingness)
* A participant chose not to step on the scale one day, so there's no weight for one day
* A sensor malfunctioned and didn't record one value
* A participant chose not to provide some form of data (like a prior life event, for example)

All of these cases lead to the data as we have it: On any given day for any given participant, we likely won't have measurements for all possible variables. However, the vast majority of machine learning methods don't work if you have missing values like this, so before thinking about fancy, niche methods, we've got two options:

    1. Delete all days that have missing values.
    2. Try and replace missing values with reasonable estimates (imputation).
    
In BUMP, almost every day has many "missing" values, so deleting all those days likely doesn't make sense. Therefore, let's focus on imputation.

### Imputation

Imputing missing values in time series data is a long-standing, well-established line of research. As an overview, there are different ways to impute:

    1. Pick a value, and swap it in for your missing values. This value could be 0 everywhere, it could be the mean of the corresponding variable, computed across the whole population or just for one participant. In pandas it might look like: df.fillna(0) or df.fillna(df.mean(0)).
    2. Compute a new value based on the missing value's neighbors (like linear interpolation, imputing missing values with the last-observed values for a given feature (forward filling) or the next-observed values (back-filling). Forward and backward filling works well when we want to assume a variable doesn't change much.
    3. Estimate what the missing value should have been with a model (machine learning). This can work well if you have a large-enough dataset and can hold out some values and train a model to fill them back in.
    
### Resampling

Instead of filling in missing values, we could also resample our timeline and aggregate values. For example, when we look at the BUMP data day-by-day, there will be lots of missing values. But on a week-by-week basis, there will be a lot less missing data. So we could just take weekly averages (for example) of the real observations for each variable, then have a new time series with one value per variable per week instead of per day.

One simple way to resample BUMP-style data is through numpy: If we have a numpy array of shape (number of participants, number of timesteps, number of variables), then we can simply add a new dimension to the array, then average over it:
```
import numpy as np

num_participants = 10
num_days = 28
num_variables = 5
window_size = 7
X = np.random.randn(num_participants, num_days, num_variables)
X = X.reshape(num_participants, window_size, -1, num_variables).mean(2)
```
In this example, we *window_size* must be a divisor of num_days. This notion of reshaping an array according to some new window-size can be useful in many cases if we want to quickly break one timeline into many smaller chunks.

## Normalization

Each variable will have different natural scales. For instance, heart rate may typically be between 50 and 100, while survey scores may range from 1 to 7. To consider differently-scaled variables together, a common approach is to rescale each time series to lie within similar ranges. Intuitively, if someone's PHQ-9 score were to spike at the same time as their heart rate, it will be easier to catch if they both range from 0 to 1. This also makes it easier to train models that are more robust, especially since many ML algorithms assume your data will be normally distributed.

### min-max normalization

When variables are already similarly-scaled, we may still want them within a small, reasonable range, like between 0 and 1. If there are no outlying large or small values, this works great and preserves the signals present in the original data. The general idea is to simply rescale based on the max and min values:

$$
X := \frac{X-\min(X)}{\max(X)-\min(X)}
$$

However, when your data instance's max value is huge, it crushes all the other timesteps down to 0, effectively deleting their information.

### z-score normalization

In the presence of particularly large or small values relative to the mean, it can be helpful to use z-score normalization instead, where we force the time series values to be on average 0 with a standard deviation of 1. This maintains the distribution of your original data. z-score normalization is quite simple, we just subtract the mean of the time series and divide by the standard deviation:

$$
X := \frac{X - \text{mean}(X)}{\text{std}(X)}
$$

## Multivariate time series

Each of these preprocessing methods extends directly to multivariate time series. For isntance, when normalizing multiple variables, we often just treat each independently!

Multivariate time series are often harder to conceptualize than standard tabular, image, or text data, because there's the additional time dimension: We have repeat measurements of the same variables for the same instances. One simple way to rectify this difference is to just flatten our time series. For instance, if we have 10 participants with 15 timesteps each, for 50 variables, we could just flatten out the timesteps and variables to be of shape (10, 750). However, this introduces two related concerns:
1. The ratio of participants to features explodes, increasing the chances of overfitting and making robust signals challenging to identify.
2. Assuming measurements don't change all that much over time, the flattened features will be highly correlated. For example, we might pretend that heart rate on day 1 is one feature and heart rate on day 2 is another, but they're very similar for all participants. This, too, increases the likelihood of overfitting.

While flattening can work sometimes in practice, avoiding flattening can be helpful, too. When predicting outcomes, we could, for instance, use methods that take in one vector of variables at a time, then predicts one new outcome, sharing parameters over time, like Vector Autoregression (VAR). Still, the most basic forms will assign one weight per timestep per variable within some window, so regularizing the weights on any models will be extremely important: If we have 750 features but only assign non-zero weights to 10 of them, our chances of overfitting are much lower.